In [1]:
# WHEN CREATING DATA, NEED TO SHIFT INPUT TOKEN INDEXES BY ONE!!!
# 1. Try with real data
# 2. Figure out if this is working the way we expect it to work or not
# 3. Check out all TODO :
# WEIGHT SHARING!!!!!!

In [2]:
# Not implemented in Autograd
# x .+= something -> does not work with Autograd
# x = x .+ something -> works

# Not implemented in Knet
# getindex for 3d KnetArray
# Also there is a bug with std function. Inside it uses _var, which uses iterate, which fails for KnetArrays

In [3]:
# The things I've solved
# 1. bmm with 4 dims
# 2. segment_ids paddings are 0, we can't index with them
# 3. ignoring indexes with labels -1 while calculating loss
# 4. 2d matrix, 3d tensor multiplication

In [18]:
import Base: length, iterate
using Random

mutable struct PreTrainingData
    input_ids
    input_mask
    segment_ids
    mlm_labels
    nsp_labels
    batchsize
    ninstances
    shuffled
end


function PreTrainingData(input_file; batchsize=16, shuffled=true, seq_len=256)
    input_ids = []
    input_mask = []
    segment_ids = []
    mlm_labels = []
    nsp_labels = []
    f = open(input_file)
    tmp = split.(readlines(f), "\t")
    for i in 1:length(tmp)
        instance = eval.(Meta.parse.(tmp[i]))
#         push!(input_ids, instance[1] .+ 1)
#         push!(input_mask, instance[2])
#         push!(segment_ids, instance[3] .+ 1)
#         push!(mlm_labels, map(x -> x != -1 ? x + 1 : x, instance[4]))
#         push!(nsp_labels, instance[5] + 1)
        push!(input_ids, (instance[1] .+ 1)[1:seq_len])
        push!(input_mask, instance[2][1:seq_len])
        push!(segment_ids, (instance[3] .+ 1)[1:seq_len])
        push!(mlm_labels, map(x -> x != -1 ? x + 1 : x, instance[4])[1:seq_len])
        push!(nsp_labels, (instance[5] + 1))
    end
    ninstances = length(input_ids)
    return PreTrainingData(input_ids, input_mask, segment_ids, mlm_labels, nsp_labels, batchsize, ninstances, shuffled)
end


function length(d::PreTrainingData)
    d, r = divrem(d.ninstances, d.batchsize)
    return r == 0 ? d : d+1
end

function iterate(d::PreTrainingData, state=ifelse(d.shuffled, randperm(d.ninstances), 1:d.ninstances))

    state === nothing && return nothing

    if length(state) > d.batchsize
        new_state = state[d.batchsize+1:end]
        input_ids = hcat(d.input_ids[state[1:d.batchsize]]...)
        input_mask = hcat(d.input_mask[state[1:d.batchsize]]...)
        segment_ids = hcat(d.segment_ids[state[1:d.batchsize]]...)
        mlm_labels = hcat(d.mlm_labels[state[1:d.batchsize]]...)
        nsp_labels = hcat(d.nsp_labels[state[1:d.batchsize]]...)
    else
        new_state = nothing
        input_ids = hcat(d.input_ids[state]...)
        input_mask = hcat(d.input_mask[state]...)
        segment_ids = hcat(d.segment_ids[state]...)
        mlm_labels = hcat(d.mlm_labels[state]...)
        nsp_labels = hcat(d.nsp_labels[state]...)
    end
    
    return ((input_ids, input_mask, segment_ids, mlm_labels, nsp_labels), new_state)
end

iterate (generic function with 207 methods)

In [40]:
include("model.jl")

load_from_torch_classification (generic function with 1 method)

In [34]:
mutable struct Config
    embed_size::Int
    vocab_size::Int
    ff_hidden_size::Int
    max_seq_len::Int
    seq_len::Int
    num_segment::Int
    num_heads::Int
    num_encoder::Int
    batchsize::Int
    atype
    pdrop
    attention_pdrop
    func
end

config = Config(768, 30522, 3072, 512, 256, 2, 12, 12, 2, KnetArray{Float32}, 0.1, 0.1, gelu)

Config(768, 30522, 3072, 512, 256, 2, 12, 12, 2, KnetArray{Float32,N} where N, 0.1, 0.1, gelu)

In [35]:
input_file = "/scratch/users/omutlu/hyperpartisan/data_20181122_big/pre_test_tiny.tsv" #pre_test_tiny Each line has 5 columns: input ids, input mask, segment ids, masked lm labels, next sentence label
dtrn = PreTrainingData(input_file, batchsize=config.batchsize, seq_len=config.seq_len)

PreTrainingData(Any[[102, 1001, 7571, 1013, 104, 1013, 10232, 1025, 10769, 2100  …  5556, 1014, 3571, 1014, 6164, 17789, 17466, 104, 18140, 103], [102, 1001, 2097, 2117, 4842, 104, 2043, 24666, 17687, 2006  …  2617, 1011, 2044, 104, 3311, 2001, 6987, 2092, 6257, 103], [102, 1997, 104, 25043, 8820, 2863, 1013, 2152, 2232, 3037  …  2009, 1013, 2046, 2098, 2023, 1038, 3790, 2007, 2024, 103], [102, 1525, 12839, 2212, 1013, 1524, 2044, 2058, 2032, 4481  …  5470, 3442, 2273, 2040, 2036, 1997, 2052, 2044, 2018, 103], [102, 4120, 1997, 24530, 2051, 1011, 3392, 2044, 2112, 5255  …  5049, 1013, 2022, 1997, 3022, 1522, 1056, 2523, 1012, 103], [102, 1001, 2688, 1012, 5642, 1012, 2261, 104, 1025, 4003  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [102, 1056, 1013, 4002, 2680, 104, 2000, 1038, 4358, 2009  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [102, 2917, 2007, 1997, 2796, 1011, 2022, 1046, 2229, 2010  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [102, 104, 1011, 2000, 27943, 15371, 1997, 10644, 2041, 2019  …  1, 1, 1, 1, 1, 

In [41]:
model = BertPreTraining(config)

BertPreTraining(Bert(EmbedLayer(Embedding(P(KnetArray{Float32,2}(768,30522))), Embedding(P(KnetArray{Float32,2}(768,512))), Embedding(P(KnetArray{Float32,2}(768,2))), LayerNormalization(P(KnetArray{Float32,1}(768)), P(KnetArray{Float32,1}(768)), 1.0e-12), 256, 0.1), Encoder[Encoder(SelfAttention(Linear3D(P(KnetArray{Float32,2}(768,768)), P(KnetArray{Float32,1}(768))), Linear3D(P(KnetArray{Float32,2}(768,768)), P(KnetArray{Float32,1}(768))), Linear3D(P(KnetArray{Float32,2}(768,768)), P(KnetArray{Float32,1}(768))), Linear3D(P(KnetArray{Float32,2}(768,768)), P(KnetArray{Float32,1}(768))), 12, 256, 768, 64, 8, 0.1, 0.1), LayerNormalization(P(KnetArray{Float32,1}(768)), P(KnetArray{Float32,1}(768)), 1.0e-12), FeedForward(Dense(Linear3D(P(KnetArray{Float32,2}(3072,768)), P(KnetArray{Float32,1}(3072))), 0.0, gelu), Linear3D(P(KnetArray{Float32,2}(768,3072)), P(KnetArray{Float32,1}(768))), 0.1), LayerNormalization(P(KnetArray{Float32,1}(768)), P(KnetArray{Float32,1}(768)), 1.0e-12)), Encoder(S

In [7]:
#model = load_from_torch(model, config.num_encoder, config.atype, torch_model)
model(dtrn)

11.008953f0

In [29]:
(x, attention_mask, segment_ids, mlm_labels, nsp_labels) = first(dtrn)
lss = @diff model(x, segment_ids, mlm_labels, nsp_labels, attention_mask=attention_mask)

T(11.1834545)

In [8]:
asd = value(model.bert.embed_layer.wordpiece.w)

768×30522 KnetArray{Float32,2}:
  0.00461547    0.00151799    0.000901209  …   0.0012588     0.000919951
 -0.00586825    0.00566539    0.00193223      -0.00394992   -0.00716168 
  0.00650557    0.00487576    0.00760175      -0.000711747  -0.0021168  
 -0.00627392    0.00571866    0.0047814       -0.00681153    0.00375465 
  0.00213985   -0.00441026   -0.00562068      -0.00748848    0.00399292 
  0.000395741   0.00470316   -0.002642     …   0.00740453   -0.00722955 
 -0.00434567   -0.00466482   -0.00266708       0.00478306    0.00655464 
 -0.00444133    0.00517291   -0.00109169      -0.00737192   -0.00713584 
  0.00524661    9.41981e-5    0.00139285       0.00638318    0.00528462 
  0.00118155   -0.00225733   -0.00128265      -0.00705595    0.00703708 
 -0.00341957   -0.00397716   -0.000586801  …  -0.00131311    0.00582492 
  0.00422543    0.00169392    0.00486032       0.00200846   -0.00626015 
  0.00269984    0.00254733    0.00113057       0.00577424    0.000879644
  ⋮                

In [9]:
dsa = value(model.mlm.linear.w)

30522×768 KnetArray{Float32,2}:
 -0.00544862    0.00320057    0.000257203  …   0.00435162   -0.00623963 
 -0.00570309    0.00346152   -0.00437561      -0.00653618    0.00719334 
  0.00607219    0.00114515   -0.00459044      -0.00510588    0.000917494
 -0.00492966    0.000598111  -0.000402751     -0.00134538    0.00232236 
  0.00697437   -0.00417384    0.00664084       0.00608568   -0.00536711 
 -0.000193653   0.00794645   -0.0052714    …   0.00284943   -0.00711361 
  0.00724944   -0.00108576    0.0018597       -0.00493013    0.00289999 
  0.000102509  -0.00658922    0.00309253      -0.00783502   -0.00174047 
 -0.00100046    0.000146513   0.00687421       0.00205391    0.000753212
 -0.00550281    2.90573e-5    0.000389272      0.000507431   0.00590259 
 -0.000906374   0.0054985    -0.000317218  …  -0.0042433    -0.00263875 
  0.00247484    0.000509544   0.00199289       0.000101284  -0.007735   
  0.00701112   -0.00150363   -0.00439735      -9.49757e-5    0.00328095 
  ⋮                

In [10]:
Knet.mean(asd .== permutedims(dsa, (2,1)))

4.2660485f-8

In [11]:
qwe = value(model.nsp.linear.w)

2×768 KnetArray{Float32,2}:
 0.0193534   -0.0143639   0.00623309  …  0.0450153  0.045012   -0.05006   
 0.00504928   0.00269391  0.0201054      0.0260925  0.0387428  -0.00371928

In [12]:
function initopt!(model::BertPreTraining, optimizer="Adam()")
    for par in params(model)
        par.opt = eval(Meta.parse(optimizer))
    end
end

function mytrain!(model::BertPreTraining, x, attention_mask, segment_ids, mlm_labels, nsp_labels)
    values = []
    J = @diff model(x, segment_ids, mlm_labels, nsp_labels, attention_mask=attention_mask)
    for par in params(model)
        g = grad(J, par)
        update!(value(par), g, par.opt)
    end
    return value(J)
end

initopt!(model)

In [13]:
losses = []
for (k, (x, attention_mask, segment_ids, mlm_labels, nsp_labels)) in enumerate(dtrn)
    lss = mytrain!(model, x, attention_mask, segment_ids, mlm_labels, nsp_labels)
    push!(losses, lss)
end
lss = Knet.mean(losses)

14.013472f0

In [14]:
value(model.nsp.linear.w)

2×768 KnetArray{Float32,2}:
 0.0173686   -0.0158124   0.0110227  …  0.0451003  0.0345928  -0.0409246
 0.00703405   0.00414239  0.0153158     0.0260075  0.049162   -0.0128547

In [15]:
Knet.mean(value(model.bert.embed_layer.wordpiece.w) .== asd)

1.0f0

In [16]:
Knet.mean(value(model.mlm.linear.w) .== dsa)

1.0f0

In [23]:
# FIRST WAY
# 1:100 row vector in python -> try in python Bert and check the result
# 1:100 column vector in julia -> try in Knet and check the result
using PyCall
@pyimport torch
@pyimport numpy
torch_model = torch.load("/scratch/users/omutlu/hyperpartisan/data_20181122_big/pytorch_model.bin")

Dict{Any,Any} with 207 entries:
  "bert.encoder.layer.2.at… => PyObject tensor([[ 0.0641,  0.0921,  0.0645,  ..…
  "bert.encoder.layer.6.ou… => PyObject tensor([0.8757, 0.8678, 0.8159, 0.8294,…
  "bert.encoder.layer.3.ou… => PyObject tensor([0.8370, 0.8401, 0.8245, 0.8292,…
  "bert.encoder.layer.5.at… => PyObject tensor([[ 0.0022,  0.0240, -0.0346,  ..…
  "bert.encoder.layer.0.at… => PyObject tensor([[ 0.0209, -0.0050, -0.0087,  ..…
  "bert.encoder.layer.0.at… => PyObject tensor([[-0.0125,  0.0259, -0.0247,  ..…
  "bert.encoder.layer.4.in… => PyObject tensor([-0.0937, -0.1178, -0.1182,  ...…
  "bert.encoder.layer.4.at… => PyObject tensor([-5.8893e-03, -2.3500e-03,  2.81…
  "bert.encoder.layer.3.at… => PyObject tensor([0.9213, 0.8878, 0.8387, 0.8576,…
  "bert.encoder.layer.9.at… => PyObject tensor([0.8576, 0.8295, 0.7983, 0.7731,…
  "bert.encoder.layer.11.a… => PyObject tensor([[-0.0619, -0.0012, -0.0201,  ..…
  "bert.encoder.layer.10.a… => PyObject tensor([ 0.0225,  0.0150,  0.0542,  0

In [90]:
torch.matmul(torch_tensor[:unsqueeze](0), torch_model["bert.encoder.layer.5.attention.self.value.weight"][:cpu]())

PyObject tensor([[ 2.8889e+02, -4.4592e+02,  1.6673e+02, -1.2045e+03,  2.1487e+02,
         -3.1990e+02, -2.1644e+02,  6.6723e+01,  4.5352e+02, -4.5058e+01,
         -1.8131e+02, -1.0027e+03, -5.6847e+02, -2.4317e+02,  3.0161e+02,
          3.1995e+02, -6.8638e+02, -5.4568e+02, -1.1507e+02, -1.0731e+01,
          1.0854e+02,  1.5143e+01,  4.5979e+01,  3.6129e+02,  6.8772e+01,
          2.9516e+01,  2.6505e+02,  8.3867e+02, -6.1576e+02,  1.5108e+02,
         -1.3967e+02,  4.2495e+01, -2.2941e+02,  1.0954e+02, -1.9463e+02,
          1.2408e+02, -3.8016e+02,  3.6273e+02,  5.1584e+01,  5.6365e+02,
         -6.4561e+02, -1.1862e+02,  1.7012e+02,  2.3527e+02, -2.4004e+02,
          3.8593e+01, -2.8113e+02, -5.6259e+02,  6.8088e+01,  1.8635e+02,
          5.5222e+02, -4.1687e+02, -3.2281e+01,  4.2259e+01,  2.2871e+02,
          4.6529e+02, -4.0624e+02,  3.7479e+01, -2.7199e+02, -4.6029e+01,
          3.9952e+02, -1.2753e+02,  5.3966e+02, -7.3936e+02,  1.0183e+03,
         -1.7781e+02, -2.5694

In [101]:
torch_model["cls.seq_relationship.weight"][:cpu]()[:numpy]()

2×768 Array{Float32,2}:
 -0.0210782  -0.0298697  -0.0114933   0.0265236  …  -0.0291932   0.00926572
  0.0132354   0.0372161   0.0114068  -0.0150693      0.035094   -0.0266885 

In [91]:
permutedims(torch_model["bert.encoder.layer.5.attention.self.value.weight"][:cpu]()[:numpy](), (2,1)) * Array{Float32}(reshape(collect(1:768).*1.0, (768,1)))

768×1 Array{Float32,2}:
   288.88898 
  -445.91956 
   166.7278  
 -1204.4648  
   214.86862 
  -319.90027 
  -216.44069 
    66.7233  
   453.52155 
   -45.058266
  -181.30641 
 -1002.7122  
  -568.46985 
     ⋮       
   -71.98135 
   460.47424 
  -315.56506 
  -146.51422 
   173.94786 
  -516.8409  
   464.60254 
  -178.05414 
  -260.75806 
  -446.14297 
   554.69507 
   347.88165 

In [48]:
torch_tensor = torch.tensor(numpy.array(1:768), dtype=torch.float32)

PyObject tensor([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,  12.,
         13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.,  24.,
         25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,  34.,  35.,  36.,
         37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,  45.,  46.,  47.,  48.,
         49.,  50.,  51.,  52.,  53.,  54.,  55.,  56.,  57.,  58.,  59.,  60.,
         61.,  62.,  63.,  64.,  65.,  66.,  67.,  68.,  69.,  70.,  71.,  72.,
         73.,  74.,  75.,  76.,  77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,
         85.,  86.,  87.,  88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,
         97.,  98.,  99., 100., 101., 102., 103., 104., 105., 106., 107., 108.,
        109., 110., 111., 112., 113., 114., 115., 116., 117., 118., 119., 120.,
        121., 122., 123., 124., 125., 126., 127., 128., 129., 130., 131., 132.,
        133., 134., 135., 136., 137., 138., 139., 140., 141., 142., 143., 144.,
        145., 146., 147., 148.,

In [36]:
torch_tensor[:unsqueeze](0)[:size]()

(1, 768)

In [27]:
torch.matmul(torch_tensor,)

PyObject tensor([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
         15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,
         29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,
         43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
         57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,
         71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,
         85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,
         99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
        113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126,
        127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140,
        141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154,
        155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175, 176, 177, 17

In [37]:
include("model.jl")

load_from_torch_classification (generic function with 1 method)

In [42]:
model = load_from_torch_pretraining(model, config.num_encoder, config.atype, torch_model)

BertPreTraining(Bert(EmbedLayer(Embedding(KFloat32[0.00618323 -0.0169476 … -0.0499978 -0.0177459; -0.0568268 -0.0541772 … -0.0468287 -0.0793887; … ; -0.0306487 -0.0289911 … -0.0284732 -0.0283612; -0.0130648 -0.0109822 … -0.0201913 0.0571289]), Embedding(KFloat32[0.0223904 -0.00928706 … 0.01999 0.00243514; -0.00443635 -0.0165968 … -0.00554573 -0.021461; … ; -0.00410777 0.0431863 … -0.0115938 0.0253736; 0.0127998 -0.019281 … -0.0258449 -0.0716339]), Embedding(KFloat32[0.00328108 -0.00180102; 0.00675674 0.00141564; … ; -0.00558636 -0.00243301; -0.00825712 -0.00647022]), LayerNormalization(KFloat32[0.921038, 0.892587, 0.840731, 0.872586, 0.910665, 0.903145, 0.928933, 0.91257, 0.954167, 0.805993  …  0.868524, 0.781676, 0.918081, 0.889764, 0.861143, 0.908655, 0.903872, 0.783843, 0.878898, 0.962423], KFloat32[-0.0262504, -0.0262299, 0.0461384, 0.097543, -0.0358597, -0.0405648, 0.0265303, -0.032672, -0.00250848, -0.00376075  …  -0.0107369, -0.0639957, -0.0192386, -0.0656198, -0.0321019, 0.0077

In [43]:
model(dtrn)

1.5112249f0

In [141]:
torch_model["cls.seq_relationship.weight"][:cpu]()[:numpy]()

2×768 Array{Float32,2}:
 -0.0210782  -0.0298697  -0.0114933   0.0265236  …  -0.0291932   0.00926572
  0.0132354   0.0372161   0.0114068  -0.0150693      0.035094   -0.0266885 

In [144]:
torch_model["cls.seq_relationship.weight"][:cpu]()[:numpy]()[end:-1:1,:]

2×768 Array{Float32,2}:
  0.0132354   0.0372161   0.0114068  -0.0150693  …   0.035094   -0.0266885 
 -0.0210782  -0.0298697  -0.0114933   0.0265236     -0.0291932   0.00926572

In [136]:
Knet.log(30522) + Knet.log(2)

11.019350194610766

In [229]:
# Check if there is weight sharing
torch_model["cls.predictions.decoder.weight"][:cpu]()[:numpy]() == torch_model["bert.embeddings.word_embeddings.weight"][:cpu]()[:numpy]()

true

In [ ]:
# Unimportant stuff after this part

In [3]:
#=
asd = randn(2,3,4,5)
dsa = randn(3,2,4,5)
@time begin
    qwe = zeros(2,2,4,5)
    for i in 1:5
        qwe[:, :, :, i] = bmm(asd[:, :, :, i], dsa[:, :, :, i])
    end
end
=#

  1.480823 seconds (3.64 M allocations: 163.294 MiB, 4.27% gc time)


In [4]:
# @time qwe2 = reshape(bmm(reshape(asd, (2,3,20)), reshape(dsa, (3,2,20))), (2,2,4,5))

  0.064232 seconds (184.66 k allocations: 9.384 MiB)


2×2×4×5 Array{Float64,4}:
[:, :, 1, 1] =
  0.115702   0.84845
 -1.28604   -1.00778

[:, :, 2, 1] =
 -0.117182   0.363062
  4.44089   -0.717041

[:, :, 3, 1] =
 -0.733266  -1.53797
 -1.17601   -1.41673

[:, :, 4, 1] =
 0.791245   -0.0872022
 0.0146814   0.0400762

[:, :, 1, 2] =
 -1.13423  0.766976
  2.13047  0.910996

[:, :, 2, 2] =
 1.41911    2.82969 
 0.716323  -0.676642

[:, :, 3, 2] =
  1.1102      -1.53823
 -0.00522836   1.55034

[:, :, 4, 2] =
 -3.75843   0.621332
  1.44112  -1.03046 

[:, :, 1, 3] =
 -0.760699  1.55247
  0.272704  3.22196

[:, :, 2, 3] =
 0.12309  2.14021
 1.64758  2.65152

[:, :, 3, 3] =
 -0.815294   1.14375 
 -0.615639  -0.989944

[:, :, 4, 3] =
 -1.07816     -1.03071 
 -0.00645643   0.314534

[:, :, 1, 4] =
 -0.366951  -0.496234
  2.55274    0.690053

[:, :, 2, 4] =
 -0.176301  -0.385411
 -0.480657   1.25897 

[:, :, 3, 4] =
  2.53924   2.71935
 -2.18906  -1.85169

[:, :, 4, 4] =
 -0.585549  -0.480815
  1.91477   -0.159747

[:, :, 1, 5] =
  0.554726  -0.8693

In [5]:
# qwe == qwe2

true

In [6]:
# I change this function in order to ignore the indexes with labels -1 in nll function. We don't need this part anymore.
#=
function findindices(y,a::AbstractArray{<:Integer}; dims=1)
    n = length(a[a.!=-1]) # ignore indexes with -1
    indices = Vector{Int}(undef,n)
    if dims == 1                   # instances in first dimension
        y1 = size(y,1)
        y2 = div(length(y),y1)
        #if n != y2; throw(DimensionMismatch()); end
        @inbounds for j=1:n
            if a[j] != -1
                indices[j] = (j-1)*y1 + a[j]
            end
        end
    elseif dims == 2               # instances in last dimension
        y2 = size(y,ndims(y))
        y1 = div(length(y),y2)
        #if n != y1; throw(DimensionMismatch()); end
        @inbounds for j=1:n
            if a[j] != -1
                indices[j] = (a[j]-1)*y1 + j
            end
        end
    else
        error("findindices only supports dims = 1 or 2")
    end
    return indices
end
=#

In [150]:
linear_model = torch.load("/scratch/users/omutlu/hyperpartisan/data_20181122_big/linear_model.pt")

Dict{Any,Any} with 4 entries:
  "dense.weight" => PyObject tensor([[ 0.0263, -0.0085,  0.0108,  ..., -0.0319,…
  "dense.bias"   => PyObject tensor([ 5.0000e-03, -9.3551e-03,  1.1515e-02, -1.…
  "final.weight" => PyObject tensor([[ 0.0118, -0.0328,  0.0302,  ...,  0.0087,…
  "final.bias"   => PyObject tensor([ 0.0352, -0.0279])

In [163]:
mutable struct Asd <: Layer
    dense::Linear
    final::Linear
end

Asd() = Asd(Linear(768,768), Linear(768,2))

function (a::Asd)(x)
    x = a.dense(x)
    display(x)
    return a.final(x)
end

Array{Float32}(reshape(collect(1:768).*1.0, (768,1)))

768×1 Array{Float32,2}:
   1.0
   2.0
   3.0
   4.0
   5.0
   6.0
   7.0
   8.0
   9.0
  10.0
  11.0
  12.0
  13.0
   ⋮  
 757.0
 758.0
 759.0
 760.0
 761.0
 762.0
 763.0
 764.0
 765.0
 766.0
 767.0
 768.0

In [164]:
asd_model = Asd()

Asd(Linear(P(Array{Float32,2}(768,768)), P(Array{Float32,1}(768))), Linear(P(Array{Float32,2}(2,768)), P(Array{Float32,1}(2))))

In [165]:
asd_model.dense.w = linear_model["dense.weight"][:cpu]()[:numpy]()
asd_model.dense.b = linear_model["dense.bias"][:cpu]()[:numpy]()
asd_model.final.w = linear_model["final.weight"][:cpu]()[:numpy]()
asd_model.final.b = linear_model["final.bias"][:cpu]()[:numpy]()

2-element Array{Float32,1}:
  0.0351763  
 -0.027943168

In [166]:
asd_model(Array{Float32}(reshape(collect(1:768).*1.0, (768,1))))
# EVERY LINEAR IS OK -> 3D dene!!!

768×1 Array{Float32,2}:
 -301.81982 
  338.263   
 -545.907   
 -253.97137 
   69.617035
 -321.05923 
  208.37355 
 -365.06345 
   41.59743 
 -103.13887 
  416.80682 
  133.91138 
 -206.91249 
    ⋮       
   26.830832
  212.42296 
 -206.62547 
  -16.682762
  315.7369  
 -324.698   
  -98.1218  
  194.63629 
   80.866425
  138.81798 
  172.95903 
 -297.8899  

2×1 Array{Float32,2}:
 -232.30048 
   20.705103

In [196]:
embed_model = torch.load("/scratch/users/omutlu/hyperpartisan/data_20181122_big/embed_model.pt")

Dict{Any,Any} with 1 entry:
  "word_embeddings.weight" => PyObject tensor([[-0.3275, -1.8525,  0.4429, -0.5…

In [197]:
asd_model = Embedding(64,10)
asd_model.w = permutedims(embed_model["word_embeddings.weight"][:cpu]()[:numpy](), (2,1))

10×64 Array{Float32,2}:
 -0.327533    0.17811   -0.504147   …   0.266249   0.948373   0.454536 
 -1.8525     -1.02624    0.730247       1.22722   -0.226292   0.698327 
  0.442899    1.36549   -0.0350884     -0.705415   1.63154   -1.31326  
 -0.565772   -0.583261  -0.231603       0.734134  -0.922967   0.946676 
 -0.739279    1.10309    1.32591       -1.47574    0.118688   0.0749492
  1.25015     0.456525   0.169865   …  -0.691988   1.15282   -0.472395 
  0.0826522   0.448301  -0.167629       1.03884   -0.976023  -0.219711 
  0.178917   -0.674799  -1.07465       -0.919094   0.728958  -0.591924 
 -0.360179   -0.685848   2.03066        1.02952   -0.555284   0.472736 
 -0.44771     1.54605   -0.573169       0.559609   0.045501  -1.58461  

In [201]:
embed_model["word_embeddings.weight"][:cpu]()[:numpy]()

64×10 Array{Float32,2}:
 -0.327533   -1.8525       0.442899   …   0.178917   -0.360179  -0.44771  
  0.17811    -1.02624      1.36549       -0.674799   -0.685848   1.54605  
 -0.504147    0.730247    -0.0350884     -1.07465     2.03066   -0.573169 
  0.386169   -0.396257     0.848314      -1.46865     1.03246    2.09795  
 -0.807839    0.548595     0.0551342     -2.4487     -1.45526    0.0600894
  0.317731    0.758683     1.92686    …  -2.25161     0.749815   0.592579 
  1.91857     0.429779    -0.270597      -0.41556    -0.132503   0.439184 
  2.27921    -0.00234205   0.411843      -0.141696    1.09544    0.479317 
  2.44634    -0.166409    -1.09532       -0.25081     1.53878   -1.6418   
  0.0465411   1.68927     -1.27962       -0.219757   -0.547628   0.022649 
  0.198385   -0.30309      1.56886    …  -0.261564   -0.800133  -0.820916 
 -0.380725    0.131117    -0.345388      -0.787145    0.322748   2.61584  
 -2.21218    -0.63758     -0.0996587      0.910331    1.38684   -0.136687 
 

In [202]:
asd = zeros(Int64, 5,2)
asd[:,1] = [11,33,16,13,55]
asd[:,2] = [44,12,14,11,43]
asd

5×2 Array{Int64,2}:
 11  44
 33  12
 16  14
 13  11
 55  43

In [203]:
asd_model(asd)

10×5×2 Array{Float32,3}:
[:, :, 1] =
  0.198385      0.259268   -0.312076    -2.21218      0.195528
 -0.30309       0.323448   -0.403771    -0.63758      1.52472 
  1.56886      -0.0422995  -0.00417269  -0.0996587    0.253939
 -0.790627     -0.121948    0.975851     0.251527    -1.90247 
 -0.000929175  -0.740367    0.00304474   0.939455     0.842466
 -1.4012        0.452447   -1.46052     -0.128605     1.40901 
 -1.0822        1.09097     0.510383    -0.00162251   1.40591 
 -0.261564      0.614126   -1.766        0.910331     0.123944
 -0.800133     -0.198369   -1.1094       1.38684     -0.525301
 -0.820916     -1.0173      0.859505    -0.136687    -0.239052

[:, :, 2] =
  1.51673    -0.380725     0.469571    0.198385      1.03165  
 -0.210208    0.131117    -0.832091   -0.30309       0.978226 
  1.6988     -0.345388    -1.30723     1.56886       1.26134  
 -0.0121622   0.386055    -0.499968   -0.790627      0.856667 
 -0.0786632  -0.578785    -0.657167   -0.000929175   0.617462 
 -0.8

In [212]:
asd = randn(10,3)
dsa = permutedims(asd, (2,1))

3×10 Array{Float64,2}:
  0.805224   0.0205297  -1.64533   …   0.420204   0.12264   -1.06714
  0.268687   1.21235    -0.197448     -1.20452    1.73878   -1.59637
 -0.226314  -0.294014   -0.957525     -0.731563  -0.629339   1.42835

In [217]:
asd[[5,2,4,9],:]

4×3 Array{Float64,2}:
  0.313378   -0.557817   0.594091 
  0.0205297   1.21235   -0.294014 
 -0.58133     0.335587  -0.0179976
  0.12264     1.73878   -0.629339 

In [218]:
dsa[:,[5,2,4,9]]

3×4 Array{Float64,2}:
  0.313378   0.0205297  -0.58133     0.12264 
 -0.557817   1.21235     0.335587    1.73878 
  0.594091  -0.294014   -0.0179976  -0.629339

In [2]:
using PyCall
@pyimport torch
@pyimport numpy
torch_model = torch.load("/scratch/users/omutlu/dl_course/project/model-64-32.pt")

Dict{Any,Any} with 201 entries:
  "bert.encoder.layer.2.at… => PyObject tensor([[ 0.0469,  0.0819,  0.0672,  ..…
  "bert.encoder.layer.6.ou… => PyObject tensor([0.8681, 0.8663, 0.8156, 0.8230,…
  "bert.encoder.layer.3.ou… => PyObject tensor([0.8274, 0.8302, 0.8236, 0.8177,…
  "bert.encoder.layer.5.at… => PyObject tensor([[ 0.0046,  0.0253, -0.0487,  ..…
  "bert.encoder.layer.0.at… => PyObject tensor([[ 0.0119,  0.0013, -0.0088,  ..…
  "bert.encoder.layer.0.at… => PyObject tensor([[-0.0169,  0.0255, -0.0256,  ..…
  "bert.encoder.layer.4.in… => PyObject tensor([-0.0872, -0.1183, -0.1181,  ...…
  "bert.encoder.layer.4.at… => PyObject tensor([-5.8873e-03, -2.3518e-03,  2.81…
  "bert.encoder.layer.3.at… => PyObject tensor([0.9145, 0.8843, 0.8339, 0.8423,…
  "bert.encoder.layer.9.at… => PyObject tensor([0.8507, 0.8258, 0.7929, 0.7660,…
  "bert.encoder.layer.11.a… => PyObject tensor([[-0.0414, -0.0008, -0.0386,  ..…
  "bert.encoder.layer.10.a… => PyObject tensor([ 0.0213,  0.0152,  0.0528,  0

In [7]:
torch_model["classifier.weight"][:cpu]()[:numpy]()

2×768 Array{Float32,2}:
 -0.00625429  -0.00539395  …  0.0189996   0.0146921    0.00993014
 -0.0177308   -0.0229973      0.0122508  -0.00701508  -0.0259408 